# 환경 설정
- 필요한 라이브러리 임포트

In [ ]:
import json
import os
import yaml

In [ ]:
dataset_path = "객체/"
save_path = "park_objects/"
images_save_path = save_path + 'images/'
labels_save_path = save_path + 'labels/'
jsons_save_path = save_path + 'jsons/'

In [ ]:
directoryNames = os.listdir(dataset_path)

In [ ]:
directoryNames

# 데이터셋 병합
- 분산되어있는 파일들을 하나의 디렉토리로 이동

In [ ]:
!cp 객체/19_벤치/*.json park_objects/jsons/
!cp 객체/25_가로등/*.json park_objects/jsons/
!cp 객체/23_화장실/*.json park_objects/jsons/
!cp 객체/9_쓰레기봉투/*.json park_objects/jsons/
!cp 객체/17_반려동물/*.json park_objects/jsons/
!cp 객체/20_공원화분/*.json park_objects/jsons/
!cp 객체/26_공원안내표지판/*.json park_objects/jsons/
!cp 객체/18_위해방지시설/*.json park_objects/jsons/
!cp 객체/14_텐트/*.json park_objects/jsons/
!cp 객체/22_휴게공간/*.json park_objects/jsons/
!cp 객체/21_쓰레기통/*.json park_objects/jsons/
!cp 객체/13_현수막/*.json park_objects/jsons/
!cp 객체/24_비석/*.json park_objects/jsons/

In [ ]:
!ls park_objects/jsons | wc -l

In [ ]:
!cp 객체/19_벤치/*.jpg park_objects/images/
!cp 객체/25_가로등/*.jpg park_objects/images/
!cp 객체/23_화장실/*.jpg park_objects/images/
!cp 객체/9_쓰레기봉투/*.jpg park_objects/images/
!cp 객체/17_반려동물/*.jpg park_objects/images/
!cp 객체/20_공원화분/*.jpg park_objects/images/
!cp 객체/26_공원안내표지판/*.jpg park_objects/images/
!cp 객체/18_위해방지시설/*.jpg park_objects/images/
!cp 객체/14_텐트/*.jpg park_objects/images/
!cp 객체/22_휴게공간/*.jpg park_objects/images/
!cp 객체/21_쓰레기통/*.jpg park_objects/images/
!cp 객체/13_현수막/*.jpg park_objects/images/
!cp 객체/24_비석/*.jpg park_objects/images/

In [ ]:
!ls park_objects/images | wc -l

In [ ]:
park_jsons = os.listdir('park_objects/jsons/')

In [ ]:
object_set = set()

for park_json_path in park_jsons:
    with open(jsons_save_path + park_json_path) as f:
        park_json = json.load(f)
    for label in park_json['category']:
        object_set.add(label['object_class_name'])

object_list = list(object_set)
object_list.sort()
print(object_list)
len(object_list)

In [ ]:
# remove useless class
# 'flame', 'food_truck', 'sit_board', 
object_list = ['banner', 'bench', 'fence', 'garbage_bag', 
               'park_headstone', 'park_info', 'park_pot', 'pet', 
               'rest_area', 'street_lamp', 'tent', 'toilet', 'trash_can']

In [ ]:
object_list.sort()
print(object_list)
len(object_list)

In [ ]:
with open(save_path+'data.yaml', 'w') as f:
    yaml.dump({'names': object_list, 'nc': len(object_list)}, f)

In [ ]:
# 데이터셋을 YOLO format으로 변환하는 코드
'''
def json_to_txt_yolo(json_path):
    # open json file
    with open(jsons_save_path + json_path) as f:
        park_json = json.load(f)
    label_file_name = json_path.split('/')[-1].replace('.json', '.txt')
    label_file_path = labels_save_path + label_file_name
    
    # save file
    f = open(label_file_path, 'w')
    image_w = park_json['images']['width']
    image_h = park_json['images']['height']
    
    # coordinate scaling
    for champ_dict in park_json['annotations']:
        champion_name = champ_dict['object_class']
        
        if champion_name not in object_list:
            continue
            
        x = abs((champ_dict['bbox'][0][0] + champ_dict['bbox'][1][0])/2)/image_w
        y = abs((champ_dict['bbox'][0][1] + champ_dict['bbox'][1][1])/2)/image_h
        w = abs(champ_dict['bbox'][0][0] - champ_dict['bbox'][1][0])/image_w
        h = abs(champ_dict['bbox'][0][1] - champ_dict['bbox'][1][1])/image_h
#         print(champion_name, x, y ,w, h)
        
        # champion_name to champion_number
        f.write(' '.join(map(str, [object_list.index(champion_name), x, y, w, h])) + '\n')
    f.close()
    
'''

In [ ]:
# 데이터셋을 클래스 비율에 맞게 YOLO format으로 변환하는 함수
# stratified data split
def rational_json_to_txt_yolo(json_path):
    # open json file
    with open(jsons_save_path + json_path) as f:
        park_json = json.load(f)
    label_file_name = json_path.split('/')[-1].replace('.json', '.txt')
    label_file_path = labels_save_path + label_file_name
    
    # save file
    f = open(label_file_path, 'w')
    image_w = park_json['images']['width']
    image_h = park_json['images']['height']
    
    # coordinate scaling
    for champ_dict in park_json['annotations']:
        champion_name = champ_dict['object_class']
        
        if champion_name not in object_list:
            continue
            
        x = abs((champ_dict['bbox'][0][0] + champ_dict['bbox'][1][0])/2)/image_w
        y = abs((champ_dict['bbox'][0][1] + champ_dict['bbox'][1][1])/2)/image_h
        w = abs(champ_dict['bbox'][0][0] - champ_dict['bbox'][1][0])/image_w
        h = abs(champ_dict['bbox'][0][1] - champ_dict['bbox'][1][1])/image_h
#         print(champion_name, x, y ,w, h)
        
        # champion_name to champion_number
        f.write(' '.join(map(str, [object_list.index(champion_name), x, y, w, h])) + '\n')
    f.close()

In [ ]:
labelTxtList = os.listdir(labels_save_path)
#len(labelTxtList)

numObjects = len(object_list)
objectDict = {}

for i in range(numObjects):
    objectDict[i] = 0
    
for txtFile in labelTxtList:
    #print(txtFile)
    
    with open(labels_save_path+txtFile, 'r') as f:
        lines = f.readlines()
        #print(lines)
        
        for line in lines:
            lineSpt = line.split(' ')
            #print(lineSpt[0])
            objectDict[int(lineSpt[0])] += 1

f.close()

sortObjectDict = sorted(objectDict.items(), key=lambda x: x[1], reverse=False)
#print(sortObjectDict)

In [ ]:
# scikit-learn 설치
# !pip install scikit-learn

# 클래스 비율에 맞게 데이터셋 분할
- json 데이터를 YOLO format으로 변환하고, 라벨을 균형적으로 조절하여 dataset 분할하는 코드

In [ ]:
trainList = []
validList = []
objectCnt = 0
for objectID, _ in sortObjectDict:
    splitObjectTxtList = []
    objectTxtList = []
    print(objectID, _)
    print(type(objectID))
    
    for txtFile in labelTxtList:
        #print(txtFile)

        with open(labels_save_path+txtFile, 'r') as f:
            lines = f.readlines()
            #print(lines)

            for line in lines:
                lineSpt = line.split(' ')
                #print(type(lineSpt[0]))
                print(lineSpt[0])

                if lineSpt[0] == str(objectID):
                    #print(txtFile)
                    #print('park_objects/images/'+txtFile.replace('.txt', '.jpg'))
                    splitObjectTxtList.append(txtFile)
                    objectTxtList.append('park_objects/images/'+txtFile.replace('.txt', '.jpg'))
                    #print(lines)
                    break

        f.close()
    
    #print("this champ has {}".format(len(splitChampTxtList)))
    
    objectCnt += len(splitObjectTxtList)
    
    #print("before labels numbers", len(labelTxtList))
    
    for splitObjectTxt in splitObjectTxtList:
        labelTxtList.remove(splitObjectTxt)
    
    #print("after labels numbers", len(labelTxtList))
    
    from sklearn.model_selection import train_test_split
    
    print(splitObjectTxtList)
    
    #trainTempList, validTempList = train_test_split(splitObjectTxtList, test_size=0.2, random_state=42)
    trainTempList, validTempList = train_test_split(objectTxtList, test_size=0.2, random_state=42)
    #print('split done')
    #print("train labels {}, valid labels {}".format(len(trainTempList), len(validTempList)))
    trainList += trainTempList
    validList += validTempList

#print('here', len(trainList), len(validList))

#print(trainList)
with open('train.txt', 'w') as f:
    f.write('\n'.join(trainList) + '\n')
    
with open('valid.txt', 'w') as f:
    f.write('\n'.join(validList) + '\n')
    
#print(objectCnt)